# Descarga de datos WeatherBench2
---

``python version = 3.11``

En este notebook se exploran los conjuntos de datos de WeatherBench2

In [6]:
# !pip install apache-beam
# !pip install git+https://github.com/google-research/weatherbench2.git
# !pip install gcsfs
# !pip install netCDF4 

In [7]:
import apache_beam  
import weatherbench2
import numpy as np
import xarray as xr

## ERA 5, resolución: 1440x721, niveles: 13, time step: 6 h
---

In [11]:
data = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
data

<xarray.Dataset>
Dimensions:                                           (time: 93544,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 13)
Coordinates:
  * latitude                                          (latitude) float32 90.0...
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float32 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 ...
    10m_v_component_of_wind                           (time, latitude, longitude) float32 ...
    10m_wind_speed                                    (time, latitude, longitude) float32 ...
    2m_dewpoint_temperature                           (time, latitude, longitude) float32 ...
    2m_temperature                                    (time, latitude, longitude) float32 ...
    above_ground                                      (time, level, latitude, longitude) float32 ...
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, latitude, longitude) float32 ...
    volumetric_soil_water_layer_2                     (time, latitude, longitude) float32 ...
    volumetric_soil_water_layer_3                     (time, latitude, longitude) float32 ...
    volumetric_soil_water_layer_4                     (time, latitude, longitude) float32 ...
    vorticity                                         (time, level, latitude, longitude) float32 ...
    wind_speed                                        (time, level, latitude, longitude) float32 ...

In [28]:
# Peso en GB
data.nbytes / 1000**3

88963.00301054

In [19]:
data.to_netcdf(r"C:\Users\gcuervo\Documents\Doctorado\DB\1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.nc")

MemoryError: Unable to allocate 362. GiB for an array with shape (93544, 721, 1440) and data type float32

La base de datos pesa ``88.963 GB`` por lo que no es posible guardarla en local.

### Selección de variables
---
Debido a que la base de datos es demasiado grande se intenta seleccionar solo unas cuantas variables ver si se puede descargar en local.

In [34]:
data_sel_vars = data[['geopotential', 'temperature', 'specific_humidity', 'u_component_of_wind', 'v_component_of_wind']]
data_sel_vars

<xarray.Dataset>
Dimensions:              (time: 93544, level: 13, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 50 100 150 200 250 ... 700 850 925 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 1959-01-01 ... 2023-01-10T18:0...
Data variables:
    geopotential         (time, level, latitude, longitude) float32 ...
    temperature          (time, level, latitude, longitude) float32 ...
    specific_humidity    (time, level, latitude, longitude) float32 ...
    u_component_of_wind  (time, level, latitude, longitude) float32 ...
    v_component_of_wind  (time, level, latitude, longitude) float32 ...

In [36]:
# Peso en GB
data_sel_vars.nbytes / 1000**3

25251.4926227

In [37]:
data_sel_vars.to_netcdf(r"C:\Users\gcuervo\Documents\Doctorado\DB\1959-2023_01_10-wb13-6h-1440x721_with_geop_temp_spehum_uvwind.nc")


MemoryError: Unable to allocate 4.59 TiB for an array with shape (93544, 13, 721, 1440) and data type float32

De la base de datos total se seleccionaron 5 variables: ``'geopotential', 'temperature', 'specific_humidity', 'u_component_of_wind', 'v_component_of_wind'``
Con un peso de: ``25251.5 GB``